<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Эксперимент Кавендиша

### Examples - Mechanics
<section class="post-meta">
By Jonas Tjemsland and Jon Andreas Støvneng
</section>
Last edited: March 15th 2018 
___


# Вступление

Закон тяготения Ньютона гласит, что сила между двумя точечными массами пропорциональна произведению их масс и обратно пропорциональна квадрату расстояния между ними. Это можно записать как

\begin{equation}
F= G\frac{m_1m_2}{r^2},
\label{eq:newton_grav}\quad(1^*)
\end{equation}

где $m_1$ и $m_2$ - массы частиц, $r$ - расстояние между ними, а $G$ - некоторая константа, известная как *гравитационная постоянная*. Сила направлена вдоль линии, пересекающей точечные массы. Можно показать, что этот закон справедлив для всех сферически симметричных распределений массы, таких как твердые шары. Приведенное выше уравнение можно даже с высокой точностью применить к гравитационному притяжению между звездами и другими небесными телами! Текущее рекомендуемое значение гравитационной постоянной CODATA составляет $(6.674 08 \pm 0.00031)\cdot 10^{-11} \text{Nm$^2$/кг$^2$}$ [1]. Это небольшое число, и измерение гравитационной постоянной, таким образом, требует чрезвычайно точного оборудования.

Первое прямое лабораторное измерение гравитационной постоянной было выполнено в 1798 году Генри Кавендишем [2].  Аппарат в первоначальном эксперименте состоял из деревянного рычага длиной 1.8 метра с двумя свинцовыми шариками диаметром около 5 см, прикрепленными с обеих сторон. Деревянный рычаг был подвешен в горизонтальном положении на проволоке длиной 1 метр. Такая установка известна как "крутильный маятник". Два больших свинцовых шара использовались для того, чтобы оказывать гравитационное притяжение на крутильный маятник, заставляя его вращаться. Из-за крутящего момента в проволоке маятник начал колебаться вокруг своего нового положения равновесия. 

Аналогичный эксперимент проводят все студенты-физики первого курса NTNU. В этом блокноте мы обсудим эксперимент Кавендиша. Мы создадим модель, описывающую колебания крутильного маятника, и, используя подгонку кривой к набору измерений, оценим период колебаний и положение равновесия. Это, в свою очередь, будет использоваться для оценки гравитационной постоянной. Теоретический раздел в значительной степени основан на лабораторном пособии, использованном в курсе механической физики FY1001 в НТНУ (см. [3]). Мы начнем с краткого обсуждения экспериментальной установки (мы ссылаемся на Руководство по лаборатории для более полного обзора эксперимента). 

# Экспериментальная Установка

Аппарат, используемый в NTNU, похож на тот, который использовал Кавендиш. Эксперимент в принципе проводится следующим образом. Два больших свинцовых шара массой $M$ установлены в положение 1, как показано на рис.1. Маятник начнет колебаться вокруг некоторого угла равновесия $\theta_1$. Лазерный луч отражается от зеркала, прикрепленного к крутильному маятнику, и попадает на линейку в положении $S(t)$. Положение на линейке записывается каждые 30 секунд. Когда система находится в состоянии покоя, свинцовые сферы устанавливаются в положение 2, заставляя маятник колебаться вокруг $\theta_2$. 

<img src="images/cavendish_setup.png"><img>

> **Рисунок 1.** *На рисунке слева приведена принципиальная схема используемого в эксперименте крутильного маятника, направленного вдоль крутильной проволоки. (1) Торсионная проволока, (2) зеркало, (3) большие свинцовые шарики, (4) маленькие свинцовые шарики, (5) лазерный луч. Всю настройку можно увидеть на рисунке справа. Позиция 1 показана сплошными линиями, а позиция 2-пунктирными линиями. Положение равновесия при отсутствии больших свинцовых шариков находится вдоль горизонтальной пунктирной линии. Цифры взяты из Лабораторного пособия по курсу F1001 Механическая физика в NTNU (ref. [2]).*

Давайте импортируем необходимые пакеты и установим общие параметры фигуры, прежде чем приступим к построению модели, описывающей затухающие колебания маятника.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
%matplotlib inline

# Set some figure parameters
newparams = {'figure.figsize': (18, 9), 'axes.grid': False,
             'lines.markersize': 6, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

## Модель

Как упоминалось во введении, Кавендиш использовал в своих измерениях крутильный маятник (также называемый крутильным балансом). Когда угол поворота (угол кручения) $\theta$ мал, его крутящий момент $\vec \tau = \vec F\cdot\vec r$ приблизительно пропорционален $\theta$. То есть,

\begin{equation}
\label{eq:torsion_pendulum}
\tau_1 =-D\theta,\quad(1)
\end{equation}

для некоторой постоянной $D$, называемой постоянной кручения. Это аналогично закону Гука для пружины.

Сопротивление воздуха объекта примерно пропорционально скорости при малых скоростях. Это отношение называется [законом Стокса](https://en.wikipedia.org/wiki/Stokes%27_law). Таким образом, крутящий момент, обусловленный сопротивлением воздуха, пропорционален угловой скорости $\dot \theta$. То есть, 

\begin{equation}
\tau_2 = -b\dot\theta.
\label{eq:air_resistance}\quad(2)
\end{equation}

Мы пренебрегаем трением в торсионной проволоке.

Второй закон Ньютона для вращения гласит,

\begin{equation}
\sum \tau_i = I\ddot\theta,
\label{eq:N2_rotation}\quad(3)
\end{equation}

где $I$ - момент инерции, в нашем случае заданный $I=2mr^2$ (две сферически симметричные массы $m$ на расстоянии $r$ от точки отсчета). Объединив уравнения (1), (2) и (3), мы получим дифференциальное уравнение

\begin{equation}
I\ddot\theta + b\dot\theta+D\theta = 0,
\label{eq:diff_eq}\quad(4)
\end{equation}

которое описывает колебания крутильного маятника. Общее решение в этом случае может быть записано как

\begin{equation}
\theta(t) = \theta_0 e^{-\alpha t}\sin\left(\omega t+\phi\right),
\label{eq:model_theta}\quad(5)
\end{equation}

где $\theta_0$ - начальная амплитуда, $\phi$ - некоторый фазовый фактор, $\omega\equiv\sqrt{\omega_0^2 - \alpha^2}$, $\alpha \equiv b/2I$ и $\omega_0 \equiv\sqrt{D/I}$. Колебание в нашем случае недодемпфированное (см., например, [4] для получения дополнительной информации), что означает, что $\omega^2 = \omega_0^2-\alpha^2>0$. Это подтверждается измеренными данными. 

В реальном эксперименте мы измеряем не $\theta$, а $S$ как положение лазерного луча на линейке (см. рис. 1). Позиция $S$ задается 

\begin{equation}
S/L=\tan\theta\approx \theta
\label{eq:S_approx}\quad(6)
\end{equation}

когда $L\ll S$ (см. упражнение 1). Поэтому мы получаем

\begin{equation}
S(t) = S_0 + A e^{-\alpha t}\sin\left(\omega t + \phi\right).
\label{eq:model_S}
\end{equation}

In [ ]:
def osc(t, S_0, A, alpha, omega, phi):
    # Модель гармонических колебаний
    return S_0 + A*np.exp(-alpha*t)*np.sin(omega*t + phi)

# Подгонка кривой

Период колебаний равен $T=2\pi/\omega$. Параметры $\phi$, $\omega$, $\alpha$, $A$ и $S_0$ можно оценить с помощью подгонки кривой модели к измеренным данным. Существует несколько способов выполнения подгонки кривой в Python. Здесь мы будем использовать [optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) из SciPy, который использует нелинейные наименьшие квадраты для подгонки функции к данным. Функция имеет три входных параметра: функция модели (*osc*), измеренные $x$-данные (*t*) и измеренные $y$-данные (*S*). Кроме того, мы будем использовать необязательный аргумент *p0*, который является начальным предположением для параметров. Функция возвращает массив оптимальных значений параметров (*params*) и соответствующую ковариационную матрицу (*cov*). Диагонали ковариационной матрицы обеспечивают дисперсию оценок параметров. Стандартное отклонение, в свою очередь, определяется квадратным корнем из дисперсий.

In [ ]:
def fit(S, t, params_init=[1,1,1,1,1]):
    """ Выполняет подгонку кривой функции osc() по точкам данных
    хранящимся в S.
    
    Параметры:
        S:           array-like vector, len(N). Измеренное смещение
                     как функция времени.
        t:           array-like vector, len(N). Время, соответствующее
                    измерениям в S.
        params_init: Первоначальное предположение для параметров.
        
    Возвращается:
        params: Параметры, которые минимизируют квадратичную ошибку (наилучшее соответствие).
        cov:    Ковариационная матрица для параметров.
    """
    try:
        params, cov = curve_fit(osc, t, S, p0=params_init)
        return params, cov
    # curve_fit вернет ошибку RuntimeError, если не сможет оценить параметры.
    except RuntimeError as err:
        print("Fit failed.")
        return params_init, np.zeros(len(params_init), len(params_init))

Мы будем использовать результаты серии измерений, проведенных некоторыми студентами НТНУ в 2014 году. Результаты в позиции 1 хранятся в файле [S1.txt](https://www.numfys.net/media/notebooks/files/S1data.txt), и позиции 2 в [S2.txt](https://www.numfys.net/media/notebooks/files/S2data.txt).

In [ ]:
# Считывание данных из файла со временем в минутах в первом столбце, время в
# секунды во втором столбце и углы в третьем столбце
# Позиция 1
data = np.loadtxt('images/S1data.txt')
t1data = data[:, 0]*60 + data[:, 1]
S1data = data[:, 2]*0.001 # m
# Position 2
data = np.loadtxt('images/S2data.txt')
t2data = data[:, 0]*60 + data[:, 1]
S2data = data[:, 2]*0.001 # m

Теперь мы готовы выполнить подгонку кривой!

In [ ]:
# Initial values for fit (educated guesses)
S0_0 = 3.50e+00 # Equilibrium line
A0 = 0.3        # Amplitude, swing
Alpha0 = 0.001  # Exponential damping coefficient for the amplitude
T0 = 600        # Swing period
phi0 = 0        # Phase angle
params_init = [S0_0, A0, Alpha0, 2*np.pi/T0, phi0]

# Fit model parameters to data
params1, cov1 = fit(S1data, t1data, params_init) # POSITION 1
err1 = np.sqrt(np.diag(cov1))
params2, cov2 = fit(S2data, t2data, params_init) # POSITION 2
err2 = np.sqrt(np.diag(cov2))

Давайте нарисуем!

In [ ]:
# Position 1
t = np.linspace(t1data[0], t1data[-1], 200)
plt.plot(t, params1[0]*np.ones(len(t)), '--', color='0.6')
plt.plot(t, osc(t, *params1), '-', color=(.5,.5,1), label='Fit position 1')
plt.plot(t1data, S1data, 'o', color=(0,0,1), label='Position 1 data')

# Position 2
t = np.linspace(t2data[0], t2data[-1], 200)
plt.plot(t, params2[0]*np.ones(len(t)), '--', color='0.6')
plt.plot(t, osc(t, *params2), '-', color=(1,.5,.5), label='Fit position 2')
plt.plot(t2data, S2data, 'o', color=(1,0,0), label='Position 1 data')

plt.xlabel('Time, (s)')
plt.ylabel('Displacement, (mm)')
plt.legend(loc='best')

plt.show()

# Гравитационная постоянная

### Получение выражения для гравитационной постоянной

Рассмотрим на данный момент только гравитационную силу $F_0$ между большими свинцовыми шарами и ближайшими маленькими шарами (то есть пренебрежем $f$ на рис. 2). Из рисунка 2 видно, что крутящий момент, обусловленный гравитационной силой $F_0$, должен быть равен крутящему моменту, обусловленному торсионной проволокой (уравнение (1)) в равновесии (когда система находится в состоянии покоя). То есть $2F_0r=D\theta_1=D\theta_2$. Используя уравнение (6), мы получаем

\begin{equation}
F_0 = \frac{D}{r}\cdot\frac{\theta_1+\theta_2}{4}\approx \frac{D}{r}\cdot\frac{S}{4L}.
\label{eq:F0}\quad(8)
\end{equation}

![Gravitational pull on the small balls](images/cavendish_forces.png)
> **Рис. 2.** *Большой шар воздействует на ближайший маленький шар гравитационной силой $F_0$. Существует также гравитационное притяжение $F_0'$ от противоположного большого шара с компонентой $f$, который уменьшает общий крутящий момент на маятнике. Рисунок взят из Лабораторного пособия по курсу F1001 Механическая физика в NTNU (ref. [2]).*

Для простоты мы предполагаем, что $\sqrt{\omega_0^2-\alpha^2}\approx \omega_0=\sqrt{D/I}$ в уравнении (5) (см. упражнение 2 и 3). Константу кручения в этом случае можно записать как

\begin{equation}
D = \frac{4\pi^2 I}{T^2}.
\label{eq:torsion_const}\quad(9)
\end{equation}

Если мы вставим уравнение (8) и (9) и $I=2mr^2$ в закон тяготения Ньютона (1*) и решим по $G$, мы получим

\begin{equation}
G = \frac{r^2}{F_0mM}=\frac{\pi^2b^2rS}{T^2LM},
\label{eq:grav_const}\quad(10)
\end{equation}

где $m$ - масса маленьких свинцовых шариков, $M$ - масса больших свинцовых шариков, а $b$ - расстояние между массами $m$ и $M$. Обратите внимание, что $m$ отменяется в конечном выражении.

Компонента $\vec f$ из $\vec F_0'$, параллельная $\vec F_0$, снижает общий крутящий момент на маятнике и дает небольшую поправку к уравнению (10). Из рисунка 2 видно, что

$$f = F_0'\cdot\frac{b}{r'},$$

где $r'=\sqrt{b^2+4r^2}$ - расстояние между шарами. Если мы вставим это в закон тяготения Ньютона (1*), то получим

$$f = G\frac{mM}{b^2+4r^2}\cdot\frac{b}{\sqrt{b^2+4r^2}}= G\frac{mM}{b^2}\cdot\frac{b^3}{(b^2+4r^2)^{3/2}}=F_0\cdot \beta,$$

где $\beta\equiv b^3/(b^2+4r^2)^{3/2}$. Таким образом, общая сила на каждом из маленьких шариков равна $F' = F_0-f = F_0(1-\beta)$. Сравнивая с уравнением (10), мы получаем скорректированное выражение для гравитационной постоянной,

\begin{equation}
G = \frac{1}{1-\beta}\cdot\frac{\pi^2b^2rS}{T^2LM}.
\label{eq:grav_const_corr}\quad(11)
\end{equation}

### Оценка гравитационной постоянной

$L$, $M$, $b$ и $r$ находят путем измерения на приборе, используемом в эксперименте. Измеренные величины, соответствующие данным, используемым в этом блокноте, определены в следующем фрагменте кода.

Мы храним эти величины в виде массивов длины 2, причем первая позиция - это значение, а вторая позиция - погрешность.

In [ ]:
b = [0.042, 0.001 ] # m
r = [0.050, 0.0001] # m
L = [2.265, 0.01  ] # m
M = [1.493, 0.002 ] # kg

Кроме того, нам нужно извлечь разницу в положениях равновесия $S=|S_{01}-S_{02}|$ и период $T$ из подгонки кривой, выполненной ранее.

Мы можем извлечь $T_1$, $T_2$, $S_{01}$ и $S_{02}$ и их стандартные отклонения из *params1*, *err1*, *params2* и *err2*. Период задается $T_i=2\pi/\omega_i$, а их отклонения связаны $\Delta T/T = \delta \omega/\omega\Rightarrow \Delta T = \Delta\omega\cdot2\pi/\omega^2$.

In [ ]:
S01 = (params1[0], err1[0])
S02 = (params2[0], err2[0])
print("S01 = ( %.4f ± %.5f ) mm"%(S01))
print("S02 = ( %.4f ± %.5f ) mm"%(S02))

T1 = (2*np.pi/params1[3], 2*np.pi/params1[3]**2*err1[3])
T2 = (2*np.pi/params2[3], 2*np.pi/params2[3]**2*err1[3])
print("T1 = ( %.2f ± %.2f ) s"%(T1))
print("T2 = ( %.2f ± %.2f ) s"%(T2))

Физическая установка одинакова в позиции 1 и позиции 2, и поэтому мы ожидаем, что $T_1\approx T_2$. Однако обратите внимание, что оценочные значения $T_1$ и $T_2$ отличаются почти на 15 секунд. Это указывает на то, что неопределенность в периоде больше, чем стандартное отклонение в подгонке. Поэтому мы будем использовать
$$T = (T_1+T_2)/2, \qquad\Delta T = |T_2-T_1|/2,$$
для период колебаний и его отклонения.

Единственная оценка погрешности, которую мы имеем для положений равновесия $S_{01}$ и $S_{02}$, - это стандартное отклонение от подгонки. Поэтому мы будем использовать
$$S = |S_{02} - S_{01}|,\qquad \Delta S = \sqrt{\Delta S_{01}^2 + \Delta S_{01}^2}.$$

Обратите внимание, что мы не учитывали ошибки в измерениях. Лучшие результаты для отклонения получаются, если эксперимент повторяется много раз.

In [ ]:
S = (abs(S01[0] - S02[0]), (S01[1]**2 + S02[1]**2)**.5)
print("S = ( %.2e ± %.1e ) mm"%(S))

T = ((T1[0] + T2[0])/2, abs(T1[0] - T2[0])/2)
print("T = ( %.2f ± %.2f ) s"%(T))

Теперь мы вставляем эти значения в уравнение (11), чтобы оценить гравитационную постоянную $G$.

In [ ]:
beta = b[0]**3*(b[0]**2+4*r[0]**2)**(-3/2.)
G = 1/(1 - beta)*np.pi**2*b[0]**2*r[0]*S[0]/(T[0]**2*L[0]*M[0])
print("G = %.2e m^3/(kg s^2)"%(G))

Величина того же порядка, что и рекомендуемое значение из CODATA $(6.674 08 \pm 0.000 31)\cdot 10^{-11} \text{Nm$^2$/kg$^2$}$. Гравитационную постоянную довольно трудно измерить, и на эксперимент влияют многие систематические ошибки [5]. Более того, измерение проводилось во время лабораторного сеанса в комнате, полной людей. Это не идеальные условия!

Поскольку мы получили отклонения во всех величинах, используемых в уравнении (11), мы также можем вычислить отклонение для $G$. Это оставлено в качестве упражнения для читателя (упражнения 4 и 5). 

## Упражнения и дальнейшая работа


1. Как мы можем избежать приближения $S/L=\tan\theta\approx \theta$? Какая дополнительная информация необходима? *Подсказка. Внимательно посмотрите на рисунок 1. Угол между линейкой и лазерным лучом на самом деле неизвестен!*
2. Используйте результат для проверки правильности приближения $\sqrt{\omega_0^2-\alpha^2}\approx \omega_0$.
3. Найдите выражение для константы кручения в уравнении (1) без аппроксимации. *Подсказка. Предположим, что информация о маленьких свинцовых шариках известна.*
4. Проанализируйте уравнение (10) - какие величины наиболее важны для точного измерения (что вызывает наибольшую ошибку в конечном выражении)?
5. Используйте отклонения $\Delta b$, $\Delta T$, $\Delta S$, $\Delta r$, $\Delta L$ и $\Delta M$ и уравнение (10), чтобы найти отклонение в оценочном значении гравитационной постоянной $\Delta G$. *Подсказка. Используйте формулу для распространения ошибок*  

## Источники

[1] Mohr, Peter J., Newell, David B. & Taylor, Barry N. (2016). CODATA recommended values of the fundamental physical constants: 2014. Rev. Mod. Phys., 88, 035009. See http://www.codata.org/.  

[2] Cavendish, Henry. Philosophical Transactions 17, 469 (1798). *Note that the goal of the original experiment was to measure the density of the Earth. However, one can express his result in terms of the gravitational constant.*

[3] Herland, Egil V., Sperstad, Iver B., Gjerden, Knut, et al.: Laboratorium i emne FY1001 mekanisk fysikk. NTNU 2016. URL: http://home.phys.ntnu.no/brukdef/undervisning/fy1001_lab/KompendiumMaster.pdf

[4] Hyperphysics.phy-astr.gsu.edu: Damped Harmonic Oscillator [Online]. http://hyperphysics.phy-astr.gsu.edu/hbase/oscda.html [retrieved Sep. 2017]  

[5] Cross, William D. Systematic Error Sources in a Measurement
of G using a Cryogenic Torsion Pendulum. University of California 2009. URL: http://www.physics.uci.edu/gravity/papers/WDCross%20thesis.pdf